# Batch inference to summarize call transcripts

## Introduction
Call center transcript summarization is a crucial task for businesses seeking to extract valuable insights from customer interactions. As the volume of call data grows, traditional analysis methods struggle to keep pace, creating a demand for scalable solutions. Batch Inference for Amazon Bedrock provides a powerful tool to address this challenge by enabling organizations to process large volumes of data efficiently.

This notebook demonstrates how to leverage batch inference for summarizing call center transcripts at scale. By processing substantial volumes of text transcripts in batches. Though we are using example of call transcript summarization here, you can really apply this to any other use case that does not need a real time output.

## Prerequisites

Before you begin, ensure that you have the following prerequisites in place:
1. Updated boto3 to 1.35.1 or greater version
2. Have raw data stored in S3 bucket as 
3. Permissions to invoke `create_model_invocation_job` API. Refer to the documentation to learn about [required permissions for batch inference job](https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-permissions.html).
4. Permission to read and write data on Amazon S3 bucket.
5. Call transcript dataset:
* This notebook was built using synthetic call transcripts in `.txt` files. If you want to try it with your own dataset, upload your call transcripts to an Amazon S3 bucket in `.txt` format. Each text file in the S3 bucket should contain only one call transcript.
* If you do not have a dataset but want to try out Batch Inference for Amazon Bedrock, you can use the synthetic call data available [here](https://github.com/aws-samples/amazon-bedrock-samples/batch-inference/call-transcript-summarization/synthetic_call_transcript_data.zip). You need to unzip the data, and then upload the `.txt` files to an S3 bucket to use the notebook below.

In [ ]:
# upgrade boto3 
%pip install --upgrade pip
%pip install boto3 --upgrade

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import boto3
import json
import os
from datetime import datetime
import time

# Bedrock client for batch inference job
bedrock = boto3.client(service_name="bedrock")

# Create an S3 client
s3 = boto3.client('s3')

# # Set the S3 bucket name and prefix for the text files
bucket_name = '<your-s3-bucket-name>'
raw_data_prefix = '<raw-data-prefix>' + '/'
out_prefix = '<prefix-for-batch-output>'

# Batch API parameters:
role_Arn = "<ARN of the role with permissions to invoke batch api for amazon bedrock>"
model_input_summary_prefix = '<input-prefix>'
jobName = '<name-of-batch-job>'
model_id = 'anthropic.claude-3-haiku-20240307-v1:0' # or use other model

# Prepare data for the batch inference:
## Data Preparation

Before initiating a batch inference job for call center transcript summarization, it's crucial to properly format and upload your data to an S3 bucket. Learn more about data format requirments in our [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-data.html).

### Formatting Input Data

The input data should be in JSONL format, with each line representing a single transcript for summarization. Each line in your JSONL file should follow this structure:

```json
{"recordId": "11 character alphanumeric string", "modelInput": {JSON body}}
```

Here, `recordId` is an 11-character alphanumeric string, working as a unique identifier for each entry. If you omit this field, the batch inference job will automatically add it in the output.

The format of the `modelInput` JSON object should match the body field for the model you are using in the `InvokeModel` request. For example, if you're using the Anthropic Claude 3 model on Amazon Bedrock, you should use the MessageAPI, and your model input might look like the following:

```json
{"recordId": "CALL0000001", 
 "modelInput": {
     "anthropic_version": "bedrock-2023-05-31", 
     "max_tokens": 1024,
     "messages": [ { 
           "role": "user", 
           "content": [{"type":"text", "text":"{<your-prompt-for-summarization>}: {<your-transcript>}" }] }],
      }
}
```

### Generating Model Inputs

The `prepare_model_inputs` function reads the input text files from an Amazon S3 bucket, generates unique record IDs, and prepares the model inputs according to the Anthropic Claude 3 model format.

In [ ]:
def prepare_model_inputs(bucket_name, prefix):
    # Initialize the model_inputs list
    model_inputs = []

    # List all text files in the S3 bucket
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    # Process each text file
    for obj in response.get('Contents', []):
        # Get the file path
        file_path = obj['Key']

        # Read the file content
        file_obj = s3.get_object(Bucket=bucket_name, Key=file_path)
        file_content = file_obj['Body'].read().decode('utf-8')

        # Generate a unique record ID
        record_id = str(int(datetime.now().timestamp())) # you can replace this with your own logic

        # Prepare the input text for the Anthropic API
        input_text = f"""Write an accurate gender-neutral summary of the following text without adding preamble or 
                        additonal information thats not present in the original text: {file_content}"""

        # Define the request body for the Anthropic API
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "messages": [{"role": 'user',
                           "content": [
                               {'type': 'text',
                                'text': input_text}]
                           }],
            "max_tokens": 300,
            "temperature": 0.1,
            "top_p": 0.1,
            "top_k": 100,
        }

        # Prepare the model input
        model_input = {
            "recordId": record_id,
            "modelInput": body
        }

        # Append the model input to the list
        model_inputs.append(model_input)

    return model_inputs

In [ ]:
%%time
# prepare data for batch inference:
model_input_jsonl = prepare_model_inputs(bucket_name, raw_data_prefix)

### Writing to JSONL File

The `write_jsonl` function takes a list of data (in this case, the list of model inputs) and a file path, and writes the data to a local JSONL file.

For each item in the data list, the function converts the item to a JSON string using `json.dumps` and writes it to the file, followed by a newline character.

In [ ]:
def write_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for item in data:
            json_str = json.dumps(item)
            file.write(json_str + '\n')

In [ ]:
# Write model inputs to a jsonl file
filename = 'batch-' + str(int(datetime.now().timestamp())) + '.jsonl'
write_jsonl(model_input_jsonl, f'{filename}')

### Uploading to Amazon S3

The `upload_to_s3` function uploads a file or directory to an AWS S3 bucket. It takes three arguments:

1. `path`: The path to the file or directory to be uploaded.
2. `bucket_name`: The name of the S3 bucket.
3. `bucket_subfolder` (optional): The name of the subfolder within the S3 bucket where the prepared data should be uploaded.

In [ ]:
def upload_to_s3(path, bucket_name, bucket_subfolder=None):
    # check if the path is a file
    if os.path.isfile(path):
        # If the path is a file, upload it directly
        object_name = os.path.basename(path) if bucket_subfolder is None else f"{bucket_subfolder}/{os.path.basename(path)}"
        try:
            s3.upload_file(path, bucket_name, object_name)
            print(f"Successfully uploaded {path} to {bucket_name}/{object_name}")
            return True
        except Exception as e:
            print(f"Error uploading {path} to S3: {e}")
            return False
    elif os.path.isdir(path):
        # If the path is a directory, recursively upload all files within it
        for root, dirs, files in os.walk(path):
            for file in files:
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, path)
                object_name = relative_path if bucket_subfolder is None else f"{bucket_subfolder}/{relative_path}"
                try:
                    s3.upload_file(file_path, bucket_name, object_name)
                    print(f"Successfully uploaded {file_path} to {bucket_name}/{object_name}")
                except Exception as e:
                    print(f"Error uploading {file_path} to S3: {e}")
        return None
    else:
        print(f"{path} is not a file or directory.")
        return None

In [ ]:
# uploads the data from local to S3 bucket for batch inference
upload_to_s3(path=f"/home/ec2-user/SageMaker/{filename}", 
             bucket_name=bucket_name, 
             bucket_subfolder=model_input_summary_prefix)

## Creating the Batch Inference Job

Once the data is prepared and uploaded to an Amazon S3, you can create the batch inference job.

### Configuring Input and Output Data

Before submitting the batch inference job, you need to configure the input and output data locations in Amazon S3. This is done using the `inputDataConfig` and `outputDataConfig` parameters.

The `inputDataConfig` specifies the Amazon S3 URI where the prepared input data (JSONL file) is stored and, the `outputDataConfig` specifies the Amazon S3 URI where the processed output data will be stored by the batch inference job.

In [ ]:
inputDataConfig=({
    "s3InputDataConfig": {
        "s3Uri": f"s3://{bucket_name}/{model_input_summary_prefix}/{filename}"
    }
})

outputDataConfig=({
    "s3OutputDataConfig": {
        "s3Uri": f"s3://{bucket_name}/{model_input_summary_prefix}/{output_prefix}/"
    }
})

In [ ]:
inputDataConfig, outputDataConfig

### Submitting the Batch Inference Job

To submit the batch inference job, you use the `create_model_invocation_job` API from the Amazon Bedrock client. This API requires the following parameters:

- `roleArn`: The Amazon Resource Name (ARN) of the IAM role with permissions to invoke the batch inference API for Amazon Bedrock.
- `modelId`: The ID of the model you want to use for batch inference (e.g., `anthropic.claude-3-haiku-20240307-v1:0`).
- `jobName`: A name for your batch inference job.
- `inputDataConfig`: The configuration for the input data, as defined in the previous step.
- `outputDataConfig`: The configuration for the output data, as defined in the previous step.

The API call returns a response containing the ARN of the submitted batch inference job.

In [ ]:
response=bedrock.create_model_invocation_job(
    roleArn=roleArn,
    modelId=model_id,
    jobName=jobName,
    inputDataConfig=inputDataConfig,
    outputDataConfig=outputDataConfig
)

### Monitoring Job Status

After submitting the batch inference job, you can monitor its status using the `get_model_invocation_job` API from the Amazon Bedrock client. This API requires the `jobIdentifier` parameter, which is the ARN of the submitted job.

In [ ]:
jobArn = response.get('jobArn')
job_id = jobArn.split('/')[1]

print(jobArn)

status = ''
while status not in ['Completed', 'Failed']:
    job_response = bedrock.get_model_invocation_job(jobIdentifier=jobArn)
    status = job_response['status']
    if status == 'Failed':
        print(job_response)
    elif status == 'Completed':
        print(datetime.now(), ": ", status)
        break
    else: 
        print(datetime.now(), ": ", status)
        time.sleep(300)

## Retrieving and Analyzing Output

When your batch inference job is complete, Amazon Bedrock creates a dedicated folder in the specified S3 bucket, using the job ID as the folder name. This folder contains a summary of the batch inference job, along with the processed inference data in JSONL format.

### Accessing and Understanding Output Format

The output files contain the processed text, observability data, and the parameters used for inference. The format of the output data will depend on the model you used for batch inference. The notebook provides an example of how to access and process this information from the output JSONL file for Anthropic Claude 3 models.

Additionally, in the output location specified for your batch inference job, you'll find a `manifest.json.out` file that provides a summary of the processed records. This file includes information such as the total number of records processed, the number of successfully processed records, the number of records with errors, and the total input and output token counts.

In [ ]:
# Set the S3 bucket name and prefix for the text files. 
# Last part in the path is the batch job's job id
prefix = f"{model_input_summary_prefix}/{output_prefix}/{job_id}/"

# Initialize the list
output_data = []

# Read the JSON file from S3
try:
    object_key = f"{prefix}{filename}.out"
    response = s3.get_object(Bucket=bucket_name, Key=object_key)
    json_data = response['Body'].read().decode('utf-8')
    
    # Process the JSON data
    for line in json_data.splitlines():
        data = json.loads(line)
        
        output_entry = {
            'request_id': data['recordId'],
            'output_text': data['modelOutput']['content'][0]['text'],
            'observability': {
                'input_tokens': data['modelOutput']['usage']['input_tokens'],
                'output_tokens': data['modelOutput']['usage']['output_tokens'],
                'model': data['modelOutput']['model'],
                'stop_reason': data['modelOutput']['stop_reason'],
                'request_id': data['recordId'],
                'max_tokens': data['modelInput']['max_tokens'],
                'temperature': data['modelInput']['temperature'],
                'top_p': data['modelInput']['top_p'],
                'top_k': data['modelInput']['top_k']
            }
        }
        output_data.append(output_entry)
    print(f"Successfully read {len(output_data)} JSON objects from S3.")
except Exception as e:
    print(f"Error reading JSON file from S3: {e}")
    
print("sample output:")
print(output_entry)

### Integrating with Existing Workflows

After retrieving the processed output data, you can integrate it into your existing workflows or analytics systems for further analysis or downstream processing. For example, you could:

- Store the summarized transcripts in a database for easy access and querying.
- Perform sentiment analysis or topic modeling on the summarized transcripts to gain additional insights.
- Categorize the summarizes into actionable business buckets and develop anomaly detection.
- Develop dashboards or reports to visualize and analyze the summarized data.

The specific integration steps will depend on your existing workflows and systems, but the processed output data from the batch inference job can be easily incorporated into various data pipelines and analytics processes.

## Conclusion

The notebook covers the entire process, from data preparation and formatting to job submission, output retrieval, and integration with existing workflows. By implementing batch inference for call transcript summarization, you can streamline your analysis processes and gain a competitive edge in understanding customer needs and improving your call center operations.

Feel free to adapt and extend this notebook to suit your specific requirements, and explore other use cases where batch inference can be applied to optimize your interactions with foundation models at scale.